In [1]:
from time import time
start = time()

In [2]:
import os
GPUs = ','.join([str(i) for i in range(0,8)])
num_gpus = len(GPUs.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = GPUs

In [3]:
import dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess
import xgboost as xgb
import gc
    
def get_ip():
    cmd = "hostname --all-ip-addresses"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    IPADDR = str(output.decode()).split()[0]
    print(IPADDR)
    return IPADDR

def get_cluster():
    ip = get_ip()
    cluster = LocalCUDACluster(ip=ip,
                               #device_memory_limit='10GB', 
                               #jit_unspill=True,
                               protocol="ucx", 
                               rmm_pool_size="29GB")
    client = Client(cluster)
    return client


In [4]:
client = get_cluster()
client

10.120.104.12


2023-04-13 18:48:43,394 - distributed.comm.ucx - WARNING - A CUDA context for device 0 (b'GPU-67be321d-9012-d150-fdc9-402004163697') already exists on process ID 17975. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
2023-04-13 18:48:44,865 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space-10093/worker-tt3h_3jx', purging
2023-04-13 18:48:44,865 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 18:48:44,865 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 18:48:44,865 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 18:48:44,865 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 18:48:44,865 - distributed.preloading -

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://10.120.104.12:8787/status,
Dashboard: http://10.120.104.12:8787/status,Workers: 8
Total threads: 8,Total memory: 1.48 TiB
Status: running,Using processes: True
Comm: ucx://10.120.104.12:38705,Workers: 8
Dashboard: http://10.120.104.12:8787/status,Total threads: 8
Started: Just now,Total memory: 1.48 TiB
Comm: ucx://10.120.104.12:40351,Total threads: 1
Dashboard: http://10.120.104.12:33863/status,Memory: 188.82 GiB
Nanny: ucx://10.120.104.12:49889,


In [5]:
%%time

users = dask_cudf.read_parquet('/raid/otto/Otto-Comp/pqs/train_v152_*.pq')

CPU times: user 902 ms, sys: 597 ms, total: 1.5 s
Wall time: 1.36 s


In [6]:
VER = 186
USE = 'clicks'

In [7]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [8]:
%%time

dtrain = xgb.dask.DaskQuantileDMatrix(client, users[FEATURES], users['clicks'])
del users

CPU times: user 2.26 s, sys: 664 ms, total: 2.93 s
Wall time: 14.5 s


In [9]:
gc.collect()

122

In [10]:
FOLDS = 5
SEED = 42

LR = 0.1

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':LR, 
    'subsample':0.7,
    'colsample_bytree':0.5, 
    'eval_metric':'map',
    'objective':'binary:logistic',
    'scale_pos_weight':8,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [11]:
%%time

output = xgb.dask.train(
        client,
        xgb_parms,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train")],
    )

/raid/data/mambaforge/envs/rapids-23.04/lib/python3.10/site-packages/xgboost/dask.py:856: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
INFO:distributed.scheduler:Receive client connection: Client-worker-8ab6f92b-da66-11ed-8760-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:38705
INFO:distributed.scheduler:Receive client connection: Client-worker-8ab6eee8-da66-11ed-875e-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:38705
INFO:distributed.scheduler:Receive client connection: Client-worker-8ab7bfdf-da66-11ed-876e-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:38705
INFO:distributed.scheduler:Receive client connection: Client-worker-8b213c9b-da66-11ed-8766-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:38705
INFO:distributed.scheduler:Receive client connecti

[0]	train-map:0.16602
[1]	train-map:0.17584
[2]	train-map:0.17556
[3]	train-map:0.17394
[4]	train-map:0.18268
[5]	train-map:0.18340
[6]	train-map:0.18293
[7]	train-map:0.18310
[8]	train-map:0.18449
[9]	train-map:0.18444
[10]	train-map:0.18537
[11]	train-map:0.18664
[12]	train-map:0.18612
[13]	train-map:0.18580
[14]	train-map:0.18605
[15]	train-map:0.18635
[16]	train-map:0.18645
[17]	train-map:0.18702
[18]	train-map:0.18742
[19]	train-map:0.18792
[20]	train-map:0.18860
[21]	train-map:0.18929
[22]	train-map:0.18969
[23]	train-map:0.18995
[24]	train-map:0.18997
[25]	train-map:0.19013
[26]	train-map:0.19014
[27]	train-map:0.19119
[28]	train-map:0.19144
[29]	train-map:0.19161
[30]	train-map:0.19194
[31]	train-map:0.19222
[32]	train-map:0.19280
[33]	train-map:0.19314
[34]	train-map:0.19391
[35]	train-map:0.19433
[36]	train-map:0.19454
[37]	train-map:0.19578
[38]	train-map:0.19605
[39]	train-map:0.19633
[40]	train-map:0.19651
[41]	train-map:0.19656
[42]	train-map:0.19669
[43]	train-map:0.1966

In [12]:
duration = time()-start
duration

83.17447137832642